링크 : https://dacon.io/competitions/official/235745/talkboard/403895?page=1&dtype=recent

## 0. 시작 전

- 주차수요 예측 AI 경진대회에 관련하여 Data Leakage 관련 문의가 많이 올라옴
- 작성자도 이 문제로 수상이 취소된 적이 있음 => 많은 분들에게 오해가 될 부분 존재

## 1. Data Leakage란?

직역하면 정보의 누설이다. 우리가 모른다고 가정된 **정보가 누설**된 경우를 의미한다.

우리가 대회 참가시 받는 데이터는 train data, 제출하는 데이터는 test data
여기서 한 가지 가정이 이루어진다.

- 우리가 알고있는 정보는 train data, 모르는 정보는 test data
> 더 직관적인 설명은 과거/현재를 train data, 우리가 알고 싶은 미래는 test data
우리는 과거와 현재를 통해 미래를 예측하는 것이 목표

미래는 개별적으로 다가온다. 따라서, 미래를 의미하는 test data들의 통계값을 파생 변수로 생성할 수 없다.

## 2. Data가 Leakage 되는 것들

### 2.1 test data의 노출

실제 예측 시에는 알 수 없는, 혹은 알아서는 안될 정보가 누출되는 경우가 있다.
=> 알 수 없는 정보가 예측에 반영되었을 때 나타나는 문제

"시간"의 개념이나 "순서"의 개념이 도입되어 있는 부분에서 자주 등장

ex) 1시간 단위로 기록된 train, test data에 "일 평균 온도" 라는 Feature 생성
=> 불가능 당장 한 시간뒤의 온도도 모르는데 어떻게 일 평균 온도를 만들까...

### 2.2 train-test 통합해서 발생하는 문제

test 데이터는 우리가 모르는 데이터이다. 
미래의 온도를 현재 시점에서 함께 통계량을 반영하는 것에 해당한다.

### 2.3 Target 값의 누수

결과 값이 하나의 파생 변수로 함께 학습되는 것
ex) 암 환자 여부 예측 시 악서 암 세포 여부라는 파생변수가 학습되면 안된다. (의학적 문제시 ㅠㅠ)

## 3. 자주 헷갈리는 부분들

### 3.1 Target Encoding(=bin encoding = effect encoding = mean encoding)

Target encoding은 encoding을 할 때 train data의 Target 값의 통계치를 활용하는 것이다.

ex) 어떤 타이타닉 데이터에서
남성의 생존률은 몇 %인가요? → 5명의 남성 중, 3명이 생존했습니다. 즉, 생존률은 60%입니다.
여성의 생존률은 몇 %인가요? → 5명의 여성 중, 2명이 생존했습니다. 즉, 생존률은 40%입니다.

그러면, 우리가 가지고 있는 train data의 통계치를 활용해 "성별"이라는 Feature를
{'남성':0.6, '여성':0.4}로 새롭게 mapping 할 수 있게 되었습니다.

그러던 중, 바다에서 새로운 사람을 발견했습니다!
그러면 모델에게 사람의 성별을 통해서 대충 예측할 수 있도록 도움을 주는 것입니다.
그 사람이 "남성이라면, train data에 기반해 예측하건데 생존률이 60%야."
그 사람이 "여성이라면, train data에 기반해 예측하건데 생존률이 40%야."
와 같이 말입니다.

=> 이런 경우 2.3에서 다룬 Target 값의 누수와 연관되어 의문
=> Target Encoding은 Data Leakage에 있어 우려가 높다.

###### 추가(1차 수정) 

Training 과정에서도 Data Leakage의 우려를 줄이기 위해서는, 
CV 과정에서도 train-valid split의 파생 변수의 target encoding을 적용해야 한다고 소개합니다.
다만, 저는 최종적으로 예측하기 위한 "test"를 위해서 training과정에서는 train data를 모두 사용해도 괜찮다고 생각합니다.
이견이 있으시면 댓글이나 새로운 토크를 올려주세요!

## 4. 주차수요 대회에서의 Data Leakage

어떤 인구 분포가 있는 어떤 지역에 어떤 주거단지를 "하나" 만들려고 할 때, 그 때 "주차 수요"가 얼마나 될지 의문을 가지는 것입니다.

그렇기 때문에 Test Data의 파생 변수를 생성하는데 있어서 우리가 알지 못하는 "미래의 주거 단지들 사이의 통계치"를 반영할 수 없습니다.

- 동일 단지코드 데이터 사이의 통계치 반영

>이 경우는 Data Leakage가 아닌 것으로 보입니다.
EX)  "동일 단지 코드 내 평균 전용면적"은 가능할 것으로 보입니다.

- 단지코드들 사이의 통계치 반영 방법

> 이 경우에는 Test Data들 사이의 통계값을 이용하면 Data Leakage로 보입니다.
EX)  test data들의 "서로 다른 단지 코드 내 평균 전용면적"은 불가능해 보입니다.

- 판단 근거

Submit 해야 하는 파일은 "단지 코드" 단위로 제출해야 합니다.
동일 주거 단지 내부의 구성은 다 마련해둔 후 (전용 면적, 전용 면적별 세대수 등), 이러한 특징을 가진 주거 단지가 있을 때, 해당 단지의 "주차 수요"를 예측하는 문제로 파악됩니다.